# MongoDB
## Install pymongo for MongoDB access

In [2]:
#import sys
#!{sys.executable} -m pip install pymongo

## Establish connection to MongoDB-Cluster

In [1]:
from pymongo import MongoClient
from pymongo.database import Database

connection_string = "PUT YOUR CONNECTION STRING HERE"
client = MongoClient(connection_string)

pubsub_db = Database(client, 'pubsub')  # connect to student database



# PubSub
## Install google-cloud-pubsub package

In [2]:
import os
GOOGLE_APPLICATION_CREDENTIALS = "/home/hdmstudent/notebooks/PATH TO CREDENTIALS JSON"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = GOOGLE_APPLICATION_CREDENTIALS

In [3]:
#!{sys.executable} -m pip install google-cloud-pubsub

## Set PubSub parameters

In [4]:
from google.cloud import pubsub_v1
project_id = "projectid"
topic_name = "topic"

## Publish a message to a topic
### Please note: this works only on the DataProc master (run here requires a json authentication!)

In [11]:
publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(project_id, topic_name)

data = "Test 1 222222 3".encode('utf-8')

future = publisher.publish(topic_path, data=data)
print('Published {} of message ID {}.'.format(data, future.result()))

Published b'Test 1 222222 3' of message ID 198158471033018.
received: b'Test 1 222222 3'
pushing to MongoDB


## Retrieve a message from a topic

In [6]:
subscriber = pubsub_v1.SubscriberClient()

topic = 'projects/{}/topics/{}'.format(project_id, topic_name)
subscription_name = 'projects/{}/subscriptions/{}'.format(project_id, topic_name)  # assumption: topic=subscription name

def callback(message):
    print("received: {}".format(message.data))
    print("pushing to MongoDB")
    mongodb_object = {"message" : message.data.split()}
    pubsub_db.messages.insert_one(mongodb_object)
    message.ack()  # acknowledge receiving of message to pubsub

future = subscriber.subscribe(subscription_name, callback)

# Read MongoDB

In [13]:
for message in pubsub_db.messages.find():
    print(message)

{'_id': ObjectId('5c5dcf3944e2f3288e9ac720'), 'message': b'Test'}
{'_id': ObjectId('5c5dcf3944e2f3288e9ac721'), 'message': b'Test'}
{'_id': ObjectId('5c5dcf4344e2f3288e9ac722'), 'message': b'Test'}
{'_id': ObjectId('5c5dcfb044e2f32a03b1c2b5'), 'message': [b'Test']}
{'_id': ObjectId('5c5dcfba44e2f32a03b1c2b6'), 'message': [b'Test', b'1', b'2', b'3']}
{'_id': ObjectId('5c5eedd8f4e83347fa85f741'), 'message': [b'Test', b'1', b'222222', b'3']}
{'_id': ObjectId('5c5eede3f4e83347fa85f742'), 'message': [b'Test', b'1', b'222222', b'3']}
